*OBjective: Find where can El Salvador citizens travel without the need of a visa, and provide recommendations based on different factors of living conditions.* 

**data from:** 

***https://visaguide.world/visa-free-countries/salvadoran-passport/***

***https://www.worlddata.info/cost-of-living.php***


Notes: found a better more resourceful website:
https://www.numbeo.com/cost-of-living/

In [1]:
import requests
import re
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.1234.56 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.google.com/',  # Replace with a commonly visited website
    'Connection': 'keep-alive',
}

r  = requests.get('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp', headers=headers)
r2 = requests.get('https://www.numbeo.com/cost-of-living/',headers=headers)
r3 = requests.get('https://visaguide.world/visa-free-countries/salvadoran-passport/',headers=headers)
column_soup  = BeautifulSoup(r.text, 'html.parser')
country_soup = BeautifulSoup(r2.text, 'html.parser')
visa_free_soup = BeautifulSoup(r3.text, 'html.parser')

data_for_column_names  = [tag for tag in column_soup.find_all(['th'])]
data_for_country_rows  = [tag for tag in country_soup.find_all(['h4','a'])]
es_visa_free_countries = [tag for tag in visa_free_soup.find_all(attrs={'ns_has-flag'})] 



In [3]:

print(f'r status code = {r.status_code}\nr2 status code = {r2.status_code}\nr3 status code = {r3.status_code}')

r status code = 200
r2 status code = 200
r3 status code = 200


In [5]:
columns_we_need           = [] # made an empty array bucket to store the scraped column names
country_rows              = [] # made an empty bucket for the countries
without_visa_countries    = [] 
evisa_countries           = []
visa_at_arrival_countries = []
visa_required_countries   = []
more_columns           = ['Visa Free Countries',
                          'eVisa Countries', 
                          'Visa on Arrival Countries',
                          'Visa Required Countries'
                          ]

# (column_names2) | adds names of columns we'll be using to empty array 
o = 0
while o < len(data_for_column_names):
    column_data = data_for_column_names[o].text
    o+=1
    columns_we_need.append(column_data)

# added more columns to insert values from Visaguide.world
start = 0
while start < len(more_columns):
    data = more_columns[start]
    columns_we_need.append(data)
    start+=1
    
# made the array bucket into a dictionary      
columns_we_need = {key: [] for key in columns_we_need}
 
# takes the soup data and gets the text from the arrays
# and stores the clean data into a new array
n = 65
while n < len(data_for_country_rows):
    country_data_in = data_for_country_rows[n].text

    if len(country_data_in) < 2:
        n+=1
        continue
    country_rows.append(country_data_in)
    n+=1

# (Country) adds values to dictionary 
country_rows = country_rows[:-11]
for value in country_rows:
    columns_we_need['Country'].append(value)


# (No Visa Countries) countries El Salvador can travel to without a visa
for i in range(len(es_visa_free_countries[:100])):
    visa_free_countries = es_visa_free_countries[i].text
    without_visa_countries.append(visa_free_countries)
for countries in without_visa_countries:
    columns_we_need['Visa Free Countries'].append(countries)


# (eVisa Countries) Countries that provide eVisa to Salvadoran Citizens
q = 100
while q < len(es_visa_free_countries[:128]):
    evisa = es_visa_free_countries[q].text
    evisa_countries.append(evisa)
    q+=1
for countries in evisa_countries:
    columns_we_need['eVisa Countries'].append(countries)

# (Visa upon Arrival) Countries that provide visa upon arrival
for s in range(128,158):
    visa_at_arrival_data = es_visa_free_countries[s].text
    visa_at_arrival_countries.append(visa_at_arrival_data)
    
for countries in visa_at_arrival_countries:
    columns_we_need['Visa on Arrival Countries'].append(countries)


# (Visa Required) Countries that require a visa to visit
for i in range(158,226):
    visa_req_data = es_visa_free_countries[i].text
    visa_required_countries.append(visa_req_data)
for countries in visa_required_countries:
    columns_we_need['Visa Required Countries'].append(countries)
    
# (Rank) Country Rank
# for ranks in columns_we_need['Country']:
    

In [6]:
columns_we_need.keys()

dict_keys(['Rank', 'Country', 'Quality of Life Index', 'Purchasing Power Index', 'Safety Index', 'Health Care Index', 'Cost of Living Index', 'Property Price to Income Ratio', 'Traffic Commute Time Index', 'Pollution Index', 'Climate Index', 'Visa Free Countries', 'eVisa Countries', 'Visa on Arrival Countries', 'Visa Required Countries'])

In [7]:
len(columns_we_need['Country'])

235

In [8]:

soup =  column_soup.find_all(['tr','td'])
soup

[<tr>
 <td style="padding-top: 5px; padding-bottom: 7px; vertical-align: bottom;">
 <a href="https://www.numbeo.com/cost-of-living/" style="border: 0; text-decoration:none;">
 <img alt="" src="https://www.numbeo.com/images/numbeo_1B3B6F.svg" style="height: 56px; padding: 0px 0px; vertical-align: bottom; border: 0;"/></a>
 </td>
 <td style="padding-left:60px; padding-top: 1px; padding-bottom: 0px; vertical-align: middle; width: 840px; text-align: left;">
 <div style="display: inline-block; width: 100%; margin-top: 10px; margin-bottom: 10px; text-align: left; vertical-align: bottom">
 <nav style="display: inline-block; vertical-align: middle;">
 <div class="standard_margin_inlineblock"> <form action="https://www.numbeo.com/common/dispatcher.jsp" id="menu_dispatch_form" method="get" style="display: inline; padding: 0; margin:0;">
 <input name="where" style="display: inline; padding:0; margin:0;" type="hidden" value="https://www.numbeo.com/quality-of-life/in/"/>
 <input class="city-selecto